## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [ ]:
!gdown 1IKfQNbkGADSFUdyPdVjQS-ucFhOiT6sM

Downloading...
From: https://drive.google.com/uc?id=1IKfQNbkGADSFUdyPdVjQS-ucFhOiT6sM
To: /content/sms_spam.csv
100% 481k/481k [00:00<00:00, 116MB/s]


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random

import nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk

from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов"

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

noise = stopwords.words('english')

df = pd.read_csv(filepath_or_buffer = "sms_spam.csv")
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Возможные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
vecs = pd.DataFrame(data=None,
                    index=['мешок n-грамм / ham','мешок n-грамм / spam','tf-idf / ham','tf-idf / spam','символьные n-граммы / ham','символьные n-граммы / spam'],
                    columns=['precision','recall','f1-score','accuracy'], dtype=None)
vecs

,precision,recall,f1-score,accuracy
мешок n-грамм / ham,NaN,NaN,NaN,NaN
мешок n-грамм / spam,NaN,NaN,NaN,NaN
tf-idf / ham,NaN,NaN,NaN,NaN
tf-idf / spam,NaN,NaN,NaN,NaN
символьные n-граммы / ham,NaN,NaN,NaN,NaN
символьные n-граммы / spam,NaN,NaN,NaN,NaN


In [ ]:
import string
import itertools
import re
from nltk.stem import WordNetLemmatizer
from string import punctuation

def cleanData(text):#функция обработки текста
    lowercase = True
    remove_stops = True
    lemmatization = True

    txt = str(text)

    txt = re.sub(r'[^A-Za-z\s]',r' ',txt) #оставляем только буквы


    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()]) #спокойно "сплитим" текст так как там остались только символы

    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w, pos='v') for w in txt.split()]) #используем лематайзер

    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in noise])  #убираем из текста стоп слова

    return txt

In [ ]:
df['text']=df['text'].apply(cleanData)
x_train, x_test, y_train, y_test = train_test_split(df.text, df['type'], train_size = 0.7)
x_train # можем пронаблюдать обработанные сообщения из x_train

2413       oh oh den muz change plan liao go back yan jiu
2309    hi neva worry bout da truth coz truth lead ur ...
2376    babe answer see maybe better reboot ym get pho...
1238           yeah sure give couple minutes track wallet
1787    per request melle melle oru minnaminunginte nu...
                              ...                        
4663                                           yeah class
3717    ok much though h friday cant wait dunno wot he...
4628    oh rite well im best mate pete go week geva lo...
3811                                   make day great day
2345                          im gloucesterroad uup later
Name: text, Length: 3891, dtype: object

In [ ]:
maxP=0
maxI=0
maxJ=0;
for i in range(1,11):
  for j in range(i,11):
    vectorizer = CountVectorizer(ngram_range=(i, j))
    vecDf = vectorizer.fit_transform(x_train)
    vectorizer.vocabulary_.items()

    clf = MultinomialNB()
    clf.fit(vecDf, y_train)

    vectorized_x_test = vectorizer.transform(x_test)
    pred = clf.predict(vectorized_x_test)
    if (maxP<float(classification_report(y_test, pred)[128:132])+float(classification_report(y_test, pred)[74:78])):
      maxP=float(classification_report(y_test, pred)[128:132])+float(classification_report(y_test, pred)[74:78])
      maxI=i;
      maxJ=j;
print(maxP,maxI,maxJ)

vectorizer = CountVectorizer(ngram_range=(maxI, maxJ))
vecDf = vectorizer.fit_transform(x_train)
vectorizer.vocabulary_.items()

clf = MultinomialNB()
clf.fit(vecDf, y_train)

vectorized_x_test = vectorizer.transform(x_test)
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))
vecs.iloc[0][0]=float(classification_report(y_test, pred)[74:78])# Парсим данные о результативностти алгоритма
vecs.iloc[1][0]=float(classification_report(y_test, pred)[128:132])
vecs.iloc[0][1]=float(classification_report(y_test, pred)[84:88])
vecs.iloc[1][1]=float(classification_report(y_test, pred)[138:142])
vecs.iloc[0][2]=float(classification_report(y_test, pred)[94:98])
vecs.iloc[1][2]=float(classification_report(y_test, pred)[148:152])
vecs.iloc[0][3]=float(classification_report(y_test, pred)[203:207])
vecs.iloc[1][3]=float(classification_report(y_test, pred)[203:207])

1.97 1 3
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1463
        spam       0.98      0.92      0.95       205

    accuracy                           0.99      1668
   macro avg       0.98      0.96      0.97      1668
weighted avg       0.99      0.99      0.99      1668



Среди разброса между 1 и 10 - граммами лучшие покозатели по предсказыванию дало использование всех от 2 до 7  n-грамм

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
maxP=0
maxI=0
maxJ=0
max_df=0
min_df=0
max_feat=0

for i in range(1,2):
  for j in range(i,2):
    for mad in np.arange(0.03,0.04,0.001):
      for mid in np.arange(0,0.0001,0.00001):
        for f in range(100,201,10):
          vectorizer = TfidfVectorizer(ngram_range=(1, 1),max_df=mad, min_df=mid, max_features=f)
          vecDf = vectorizer.fit_transform(x_train)
          vectorizer.vocabulary_.items()

          clf = MultinomialNB()
          clf.fit(vecDf, y_train)

          vectorized_x_test = vectorizer.transform(x_test)
          pred = clf.predict(vectorized_x_test)
          if (maxP<float(classification_report(y_test, pred)[128:132])+float(classification_report(y_test, pred)[74:78])):
            maxP=float(classification_report(y_test, pred)[128:132])+float(classification_report(y_test, pred)[74:78])
            maxI=i
            maxJ=j
            max_df=mad
            min_df=mid
            max_feat=f

In [ ]:
print(maxP,maxI,maxJ,max_df,min_df,max_feat)

vectorizer = TfidfVectorizer(ngram_range=(maxI, maxJ),max_df=max_df, min_df=min_df, max_features=max_feat)
vecDf = vectorizer.fit_transform(x_train)
vectorizer.vocabulary_.items()

clf = MultinomialNB()
clf.fit(vecDf, y_train)

vectorized_x_test = vectorizer.transform(x_test)
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

vecs.iloc[2][0]=float(classification_report(y_test, pred)[74:78])# Парсим данные о результативностти алгоритма
vecs.iloc[3][0]=float(classification_report(y_test, pred)[128:132])
vecs.iloc[2][1]=float(classification_report(y_test, pred)[84:88])
vecs.iloc[3][1]=float(classification_report(y_test, pred)[138:142])
vecs.iloc[2][2]=float(classification_report(y_test, pred)[94:98])
vecs.iloc[3][2]=float(classification_report(y_test, pred)[148:152])
vecs.iloc[2][3]=float(classification_report(y_test, pred)[203:207])
vecs.iloc[3][3]=float(classification_report(y_test, pred)[203:207])

1.9 1 1 0.03 0.0 190
              precision    recall  f1-score   support

         ham       0.96      0.99      0.98      1463
        spam       0.94      0.71      0.81       205

    accuracy                           0.96      1668
   macro avg       0.95      0.85      0.89      1668
weighted avg       0.96      0.96      0.96      1668



Среди разброса между 1 и 10 - граммами лучшие покозатели по предсказыванию дало использование униграмм (поэтому для дальнейшего перебора использовались только они) max_df как и min_df показывал наилучшие результаты на числах как можно меньших, но лучшое из того что перебралось выштло 0.036 и 0.0 соответственно, max_features давал хорошие результаты в диапозоне от 110 до 180, но в нашем случае получилось 130

In [ ]:
maxP=0
maxI=0
maxJ=0;
for i in range(1,11):
  for j in range(i,11):
    vectorizer = CountVectorizer(analyzer = 'char', ngram_range=(i, j))
    vecDf = vectorizer.fit_transform(x_train)
    vectorizer.vocabulary_.items()

    clf = MultinomialNB()
    clf.fit(vecDf, y_train)

    vectorized_x_test = vectorizer.transform(x_test)
    pred = clf.predict(vectorized_x_test)
    if (maxP<float(classification_report(y_test, pred)[128:132])+float(classification_report(y_test, pred)[74:78])):
      maxP=float(classification_report(y_test, pred)[128:132])+float(classification_report(y_test, pred)[74:78])
      maxI=i;
      maxJ=j;
print(maxP,maxI,maxJ)

vectorizer = CountVectorizer(ngram_range=(maxI, maxJ))
vecDf = vectorizer.fit_transform(x_train)
vectorizer.vocabulary_.items()

clf = MultinomialNB()
clf.fit(vecDf, y_train)

vectorized_x_test = vectorizer.transform(x_test)
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

vecs.iloc[4][0]=float(classification_report(y_test, pred)[74:78])# Парсим данные о результативностти алгоритма
vecs.iloc[5][0]=float(classification_report(y_test, pred)[128:132])
vecs.iloc[4][1]=float(classification_report(y_test, pred)[84:88])
vecs.iloc[5][1]=float(classification_report(y_test, pred)[138:142])
vecs.iloc[4][2]=float(classification_report(y_test, pred)[94:98])
vecs.iloc[5][2]=float(classification_report(y_test, pred)[148:152])
vecs.iloc[4][3]=float(classification_report(y_test, pred)[203:207])
vecs.iloc[5][3]=float(classification_report(y_test, pred)[203:207])

1.93 1 9
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1463
        spam       0.98      0.92      0.95       205

    accuracy                           0.99      1668
   macro avg       0.99      0.96      0.97      1668
weighted avg       0.99      0.99      0.99      1668



Среди разброса между 1 и 10 - символьными  n-граммами лучшие покозатели по предсказыванию дало использование всех n-грамм от 1 до 10

In [ ]:
vecs

,precision,recall,f1-score,accuracy
мешок n-грамм / ham,0.99,1.0,0.99,0.99
мешок n-грамм / spam,0.98,0.92,0.95,0.99
tf-idf / ham,0.96,0.99,0.98,0.96
tf-idf / spam,0.94,0.71,0.81,0.96
символьные n-граммы / ham,0.99,1.0,0.99,0.99
символьные n-граммы / spam,0.98,0.92,0.95,0.99


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
s = 'Мальчик, -который вы-жил'

result = re.findall(r'(\b(?<!-)\w{2}|\b\w-\w)', s)
print(result)

['Ма', 'вы']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
s = ' Мистер и миссис Дурсль проживали в доме номер четыре по Тисовой улице и всегда с гордостью заявляли, что они, слава богу, абсолютно нормальные люди. Уж от кого-кого, а от них никак нельзя было ожидать, чтобы они попали в какую-нибудь странную или загадочную ситуацию. Мистер и миссис Дурсль весьма неодобрительно относились к любым странностям, загадкам и прочей ерунде.'
result = re.split('\.', s,maxsplit=3)
print(result)

[' Мистер и миссис Дурсль проживали в доме номер четыре по Тисовой улице и всегда с гордостью заявляли, что они, слава богу, абсолютно нормальные люди', ' Уж от кого-кого, а от них никак нельзя было ожидать, чтобы они попали в какую-нибудь странную или загадочную ситуацию', ' Мистер и миссис Дурсль весьма неодобрительно относились к любым странностям, загадкам и прочей ерунде', '']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub('\d', 'DIG', '50 очков Гриффиндору')
print (result)

DIGDIG очков Гриффиндору


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub('http[s]?://[\w.:?/=]+.[a-z]', '', '123 https://неURL 456 https://twitter.com/elmalfoy73 789 https://www.instagram.com/elmalfoy73')
print (result)

123 https://неURL 456 73 789 73


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[А-Яа-яё\-]{3,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё. fg h")

['Слова', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
s='abc.test@gmail.com xyz@test test.first@analyticsvidhya.com first.test@rest.biz'
prog = re.compile('@[a-z_\-]+\.[a-z]{2,}')
prog.findall(s)

['@gmail.com', '@analyticsvidhya.com', '@rest.biz']